# Indebted to Nicholas Jelicic, creator of this notebook for making a Simple LR model to crack this problem
https://www.kaggle.com/njelicic/linear-regression

# Similarly indebted to Jorijn Jacko Smit, creator of this notebook for using GroupkFold to crack this problem
https://www.kaggle.com/jorijnsmit/found-the-holy-grail-grouptimeseriessplit

In [ ]:
!pip install featurewiz

In [ ]:
import featurewiz as FW

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import janestreet
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
kaggle_yes = True
import os
i = 0
files = []
if kaggle_yes:
    datapath = '/kaggle/input'
else:
    datapath = '../Ram/data_sets/Janata_Hack_TS'
for dirname, _, filenames in os.walk(datapath):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))
        print(files[i])
        i += 1 

In [ ]:
train_df = pd.read_csv(files[3],nrows=400000)
print(train_df.shape)
train_df.head(1)

In [ ]:
target = 'resp'
date_col = 'date'
id_col = 'ts_id'
ignore_cols = ['resp_1','resp_2','resp_3','resp_4']

In [ ]:
train_df = train_df.fillna(-999)
train_df.isnull().sum().sum()

In [ ]:
train_df.drop(ignore_cols,axis=1,inplace=True)

In [ ]:
preds = [x for x in list(train_df) if x not in [target,'ts_id']]
len(preds)

In [ ]:
from sklearn.preprocessing import StandardScaler


output = FW.featurewiz(orig_train, target, corr_limit=0.70,
                    verbose=2, sep=',', header=0, test_data=orig_test,
                    feature_engg='target', category_encoders='')

In [ ]:
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.fillna(-999)
    X_test = test_df[preds]
    y_preds = FW.simple_XGBoost_model(X_XGB=train_df[preds], Y_XGB=train_df[target], 
                                  X_XGB_test=X_test, 
                     modeltype='Regression', 
                     log_y=False, GPU_flag=True, 
                     scaler=StandardScaler(), enc_method='label', verbose=0)
    action = ((test_df['weight'].values * y_preds) > 0).astype('int')

    sample_prediction_df.action = action
    env.predict(sample_prediction_df)

In [ ]:
test = pd.read_csv(files[2])
test = test.fillna(-999)
print(test.shape)
test.head()

In [ ]:
y_preds1 = FW.simple_XGBoost_model(X_XGB=train_df[preds], Y_XGB=train_df[target], 
                                  X_XGB_test=test[preds], 
                     modeltype='Regression', 
                     log_y=False, GPU_flag=True, 
                     scaler=StandardScaler(), enc_method='label', verbose=0)  
y_preds1 = ((test['weight'].values * y_preds1) > 0).astype('int')
y_preds1

In [ ]:
y_preds1.mean()

In [ ]:
subm = pd.read_csv(files[0])
print(subm.shape)
subm['action'] = y_preds1
subm.head()

In [ ]:
subm.to_csv('submission.csv',index=False)